# **Détection de conducteur distrait**
### Par : KANGA OI KANGA PIERRE STEPHANE

In [ ]:
import os.path as osp
import random
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import fastai
from fastai.vision.all import *
from fastai.metrics import error_rate
fastai.__version__

# 1. Fixation de la graine dans tout le projet

In [ ]:
SEED = 2021
np.random.seed(SEED)
random.seed(SEED)

# 2. Chargeons les images avec la librairie fastai et visualisons un lot (batch)

In [ ]:
chemin_acces = '/kaggle/input/state-farm-distracted-driver-detection/'
csv_file_path = osp.join(chemin_acces, 'driver_imgs_list.csv')

df = pd.read_csv(csv_file_path) 
df.head(5)

Les données d'apprentissage sont composées d'images de conducteurs repartis en 10 classes à savoir :

*   c0: normal driving
*   c1: texting - right
*   c2: talking on the phone - right
*   c3: texting - left
*   c4: talking on the phone - left
*   c5: operating the radio
*   c6: drinking
*   c7: reaching behind
*   c8: hair and makeup
*   c9: talking to passenger

In [ ]:
def label_classe(x):
    return (df.loc[df.img==x, "classname"].values[0])

path = osp.join(chemin_acces, "imgs/train")
dls = ImageDataLoaders.from_name_func(path, get_image_files(path), valid_pct=0.25, seed=SEED, label_func=label_classe, item_tfms=Resize(224))

dls.show_batch()

# 3. Déterminons les différentes strates ou aspects de ces images

In [ ]:
by_drivers = df.groupby('subject') # 
unique_drivers = by_drivers.groups.keys() #

print('Nombre image dans le dataset train : ', df.shape[0])
# determinon le nombre de conducteurs
print('Nombre de conducteurs unique : ',len(unique_drivers)) 

Notre jeux de données train contient **22424 images**. Ces images sont celles de **26** conducteurs.

L'histogramme de la repartition des images par classe nous permet de voir une repartition équilibré du nombre d'images par classe. 

In [ ]:
classe = df.classname.value_counts()
fig = classe.plot(kind='bar')

Par contre une distribution des images par conducteurs nous presente une répartition déséquilibré.

In [ ]:
subject = df.subject.value_counts()
fig = subject.plot(kind='bar')

## Construction d'un ensemble de validation équilibré avec un ratio de 25%

Afin d'avoir un ensemble de validation équilibré, nous choissisons les conducteurs dont leur image devront faire partie uniquement de l'ensemble de validation tout en respectant la proportion des 25%.

In [ ]:
choix_conducteur = ['p081','p050','p022','p012', 'p021','p002']

def ens_validation(df, subject):
  df_return = pd.DataFrame(columns = df.columns)
  df_return["is_valid"] = ""
  for i in df["classname"].unique():
    groupe = df[df["classname"]==i]
    validation = groupe[groupe.subject.isin(subject)].assign(is_valid=True)
    train = groupe[~groupe.subject.isin(subject)].assign(is_valid=False)
    resultat = pd.concat([validation, train],ignore_index=True)
    df_return = pd.concat([df_return, resultat],ignore_index=True)
    
  return df_return

In [ ]:
df1 = ens_validation(df,choix_conducteur)

df_final = pd.DataFrame(columns = ["img","classname","is_valid"])
df_final["img"]=df1["img"]
df_final["classname"]=df1["classname"]
df_final["is_valid"]=df1["is_valid"]
df_final['img'] = df.apply(lambda x: osp.join(x.classname, x.img), axis=1)
df_final.head()

La vérification des résultats nous donne :

In [ ]:
print('Ensemble de validation : ',df_final[df_final["is_valid"]==True].shape[0], ' images')
print('Ensemble de train : ',df_final[df_final["is_valid"]==False].shape[0], ' images')

## Chargement des images avec la librairie fastai avec la methode from_df

In [ ]:
path = Path('/kaggle/input/state-farm-distracted-driver-detection/imgs/')
dls = ImageDataLoaders.from_df(df_final, path, folder='train', label_col=1, valid_col='is_valid',
                               seed=SEED, item_tfms=Resize(224))

Visualisons les images avec un lot :

In [ ]:
dls.show_batch()

## 4. Entraînons notre modèle (apprentissage par transfert) pendant 5 époques

Nous utilisons le reseau resnet18 pour notre apprentissage par transfert.

In [ ]:
reseau_18 = cnn_learner(dls, resnet18, metrics=[error_rate], model_dir="/tmp/model/")

recherchons le rythme d’apprentissage optimal fourni par fastai

In [ ]:
lr_rate = reseau_18.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
print('le rythme fourni par fastai est : ', lr_rate.minimum)

Nous entrainons maintenant notre modèle avec ce rythme d'apprentissage pendant **5 époques**

In [ ]:
reseau_18.fine_tune(epochs=5, base_lr=lr_rate.minimum, cbs=[ShowGraphCallback(),])

In [ ]:
#sauvegarde du modele
reseau_18.save("./sauvegarde_model18_1")

# 5. Entraînons notre modèle à nouveau jusqu’à la dégradation (forte ameliorementation) du loss de l’ensemble de validation. Précisons le numéro de l’époque où cette divergence du loss a débuté et la meilleure performance obtenue.

In [ ]:
reseau_18.fine_tune(epochs=15, base_lr=lr_rate.minimum, cbs=[ShowGraphCallback(),])

Nous avons une divergence a la **4e** epoque. La valeur de la meilleur performance est **0.456580** en terme de taux d'erreur

In [ ]:
# sauvegarde du modèle
reseau_18.save("./sauvegarde_model18_2")

# 6. Entraînons de nouveau notre modèle en effectuant des changements de sorte à améliorer la performance du modèle.

In [ ]:
changement = [Rotate(360, p=1, mode='bilinear'), Brightness(max_lighting=0.9, p=0.35),Contrast(max_lighting=0.4, p=0.25),
              RandomErasing(p=0.2, sl=0.0, sh=0.2, min_aspect=0.3, max_count=2), Flip(p=1), Zoom(max_zoom=1.1,p=0.1), RandomResizedCrop(224)]

dls_new = ImageDataLoaders.from_df(df_final, path, folder='train', label_col=1, valid_col='is_valid',
                               seed=SEED, batch_tfms=changement, item_tfms=Resize(224))

In [ ]:
reseau_18_new = cnn_learner(dls_new, resnet18, metrics=[error_rate], model_dir="/tmp/model/")

In [ ]:
lr_rate_new = reseau_18_new.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
print('le nouveau rythme après changement fourni par fastai est : ', lr_rate_new.minimum)

In [ ]:
reseau_18_new.fine_tune(epochs=10, base_lr=lr_rate_new.minimum, cbs=[ShowGraphCallback(),])

In [ ]:
# sauvegarde du modèle
reseau_18_new.save("./sauvegarde_model18_3")

# 7. Présentons les résultats de notre modèle et critiquons-le.

In [ ]:
interpretation = ClassificationInterpretation.from_learner(reseau_18_new)
interpretation.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interpretation.print_classification_report()

# 8. À l’aide des outils vus en cours, diagnostiquons 6 prédictions à raison de 2 prédictions pour chacune des 3 classes ayant enregistré les plus mauvaises performances. Expliquons les raisons de ces erreurs pour chaque classe. Quelles suggestions pouvez vous faire pour l’amélioration du modèle ?

In [ ]:
interpretation.most_confused(min_val=50)